In [1]:
import os
import torch
import pickle

from torchinfo import summary

/home/jose/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from models import (
    make_resnet_pretrained_classifier,
    make_slowfast_pretrained_classifier,
    make_slow_pretrained_segmenter)
from lared_laughter.constants import models_path

In [3]:
pretrained = make_resnet_pretrained_classifier()
# pretrained = make_slowfast_pretrained_classifier()

In [4]:
input = [
    torch.rand((8, 3, 8, 256, 256)), # slow
    torch.rand((8, 3, 32, 256, 256)) # fast
]

In [4]:
# summary(pretrained, input_data=input, device='cuda', depth=5)
summary(pretrained, input_size=(8, 3, 8, 256, 256), device='cuda', depth=5)

Layer (type:depth-idx)                                  Output Shape              Param #
PTVResNet                                               [8, 2]                    --
├─Net: 1-1                                              [8, 2, 1, 2, 2]           --
│    └─ModuleList: 2-1                                  --                        --
│    │    └─ResNetBasicStem: 3-1                        [8, 64, 8, 64, 64]        --
│    │    │    └─Conv3d: 4-1                            [8, 64, 8, 128, 128]      (9,408)
│    │    │    └─BatchNorm3d: 4-2                       [8, 64, 8, 128, 128]      (128)
│    │    │    └─ReLU: 4-3                              [8, 64, 8, 128, 128]      --
│    │    │    └─MaxPool3d: 4-4                         [8, 64, 8, 64, 64]        --
│    │    └─ResStage: 3-2                               [8, 256, 8, 64, 64]       --
│    │    │    └─ModuleList: 4                          --                        --
│    │    │    │    └─ResBlock: 5-1                 

In [5]:
pretrained.model.blocks

ModuleList(
  (0): ResNetBasicStem(
    (conv): Conv3d(3, 64, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
    (norm): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation): ReLU(inplace=True)
    (pool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=[0, 1, 1], dilation=1, ceil_mode=False)
  )
  (1): ResStage(
    (res_blocks): ModuleList(
      (0): ResBlock(
        (branch1_conv): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (branch1_norm): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (branch2): BottleneckBlock(
          (conv_a): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
          (norm_a): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_a): ReLU(inplace=True)
          (conv_b): Conv3d(64, 64, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1),